In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import openpyxl
from time import time, localtime, strftime

class ExplorerScreenShot:
    def __init__(self, workbook, sheet_name):
        self.workbook = openpyxl.load_workbook(filename=workbook, data_only=True)
        self.sheet_name = sheet_name
        self.sheet = self.workbook[sheet_name]

        self.options = webdriver.ChromeOptions()
        self.options.add_argument("headless")
        self.options.add_argument("--window-size=1920,1440")
        self.options.add_argument(f'user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36')
        
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=self.options)
        
        self.network_map = {
            'ETH': "https://etherscan.io/tx/",
            'ERC20': "https://etherscan.io/tx/",
            "BTC": "https://www.blockchain.com/btc/tx/",
        }


    def logger(func):
        def wrapper(*args, **kwargs):
            start = time()
            start_time = strftime("%Y-%m-%d %I:%M:%S", localtime(start))
            print(f"starts now! {start_time}")
            func(*args, **kwargs)
            end = time()
            end_time = strftime("%Y-%m-%d %I:%M:%S", localtime(end))
            print(f"completed at {end_time}, took {round(end - start, 2)} seconds")
        return wrapper
    
    @logger
    def get_screenshot(self, target_network, start = 1):
        for i in range(start, self.sheet.max_row + 1):
            network = self.sheet[f"K{str(i)}"].value
            if network in target_network:
                tx_hash = self.sheet[f"L{str(i)}"].value
                print(f"network: {network}, tx_hash: {tx_hash}")
                self.driver.implicitly_wait(6)
                self.driver.get( self.network_map[network] + tx_hash )
                self.driver.implicitly_wait(6)
                self.driver.save_screenshot(f"screenshots/{self.sheet_name}_{network}_row{i}.png")
                self.driver.implicitly_wait(6)

        self.driver.quit()

In [15]:
filename ="screenshots/OKEx Funding Account History_v5.xlsx"
es = ExplorerScreenShot(filename, "OKEx Funding 출금 History")
es.get_screenshot(["ERC20", "BTC"], 1)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/bbrick/.wdm/drivers/chromedriver/mac64/103.0.5060.24/chromedriver] found in cache


starts now! 2022-06-22 05:35:48
network: ERC20, tx_hash: 0x275512c3aa3e8fb4d3b9920d897bfd18b40cde2d57dbec8007c3fc7d41224edb
network: ERC20, tx_hash: 0x79b2741320297b8d3432faeed93ba670464284e171d9af95db1918162f68c3c4
network: ERC20, tx_hash: 0xd82136432629a7cab8dfd3647efd2fae09a801a7fb7138d767231700c08cf095
network: ERC20, tx_hash: 0x53d627576dca14fdc10957db8b3edb6adf14d06fa05e207f08921b76de91340c
network: ERC20, tx_hash: 42a32604c837703ad65c78ae13e861c6d638acb95897b49c535c1cfab1766e0e
network: ERC20, tx_hash: 0xd1bf48d2786d8fbe866f5e6ef4e7dd991cfb42de2a44207a64ba6b50c7aba591
network: ERC20, tx_hash: 0xdd9166b2588dd77e31e0cd8a5b61bfe218bbf1807cf546eccee1d368abdb5c45
network: ERC20, tx_hash: 0xc20ae6078d5afbcdd9e4c41008196253a2266e4eb33531ffcad16019eff238d7
network: ERC20, tx_hash: 0x18000afb2b2378a748cdb91ce967d1a4633cab8c506d9c2c55c853a7ae0023ea
network: ERC20, tx_hash: 0xf6497b1f991b2203418097444f598da4c1b118b4bd8cb81e73e45fc2e25171a9
network: ERC20, tx_hash: 0x4f96251792b852779503fff